In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from torch.utils.data import DataLoader
from dataset import DCASE

In [6]:
root_dir = os.getcwd() + '/ADL_DCASE_DATA/development'
dataset = DCASE(root_dir, 3)

In [7]:
loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [8]:
iterator = iter(loader)

data, label = iterator.next()

In [9]:
class CNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=128,
            kernel_size=5
        )
        self.batch1 = nn.BatchNorm2d(128)
        self.conv2 = nn.Conv2d(
            in_channels=128,
            out_channels=256,
            kernel_size=5
        )
        self.batch2 = nn.BatchNorm2d(128)
        self.pool1 = nn.MaxPool2d(
            kernel_size=5
            stride=5
        )
        self.pool2 = nn.AdaptiveMaxPool2d((4, None))
        self.fc1 = nn.Linear(4096, 15)
        
        self.initialise_layer(self.conv1)
        self.initialise_layer(self.conv2)
        
    def forward(x):
        
        
    @staticmethod
    def initialise_layer(layer):
        if hasattr(layer, "bias"):
            nn.init.zeros_(layer.bias)
        if hasattr(layer, "weight"):
            nn.init.kaiming_normal_(layer.weight)
        

SyntaxError: unexpected EOF while parsing (<ipython-input-9-e617f9de46d9>, line 25)

In [11]:
data.shape, label

(torch.Size([1, 10, 60, 150]), tensor([8]))

In [47]:
conv1 = nn.Conv2d(
    in_channels=1,
    out_channels=128,
    kernel_size=5
)
batch1 = nn.BatchNorm2d(128)
conv2 = nn.Conv2d(
    in_channels=128,
    out_channels=256,
    kernel_size=5
)
batch2 = nn.BatchNorm2d(256)
pool1 = nn.MaxPool2d(
    kernel_size=5,
    stride=5
)
pool2 = nn.AdaptiveMaxPool2d((4, None))
fc1 = nn.Linear(4096, 15)

In [38]:
x = data.view((-1, 1, data.shape[2], data.shape[3]))

In [42]:
x1 = conv1(x)
x1.shape, x.shape

(torch.Size([10, 128, 56, 146]), torch.Size([10, 1, 60, 150]))

In [43]:
x2 = batch1(x1)
x2.shape, x1.shape

(torch.Size([10, 128, 56, 146]), torch.Size([10, 128, 56, 146]))

In [44]:
x3 = pool1(F.relu(x2))
x3.shape, x2.shape

(torch.Size([10, 128, 11, 29]), torch.Size([10, 128, 56, 146]))

In [45]:
x4 = conv2(x3)
x4.shape, x3.shape

(torch.Size([10, 256, 7, 25]), torch.Size([10, 128, 11, 29]))

In [49]:
x5 = batch2(x4)
x5.shape, x4.shape

(torch.Size([10, 256, 7, 25]), torch.Size([10, 256, 7, 25]))

In [50]:
x6 = pool2(x5)
x6.shape, x5.shape

(torch.Size([10, 256, 4, 25]), torch.Size([10, 256, 7, 25]))

In [51]:
x7 = torch.flatten(x6, 1)
x7.shape, x6.shape

(torch.Size([10, 25600]), torch.Size([10, 256, 4, 25]))